In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是对[Amy Wu](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/intro-swivel.ipynb)的笔记本进行修订后的版本。

# GCP上的E2E ML：MLOps阶段6：serving：使用Vertex AI Vector Search和Swivel内置算法开始

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine_swivel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine_swivel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine_swivel.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本笔记本演示了如何使用 Vertex AI Pipelines 训练一个嵌入（embedding），使用的是子矩阵向量嵌入学习器（[Swivel](https://arxiv.org/abs/1602.02215)）。嵌入学习器的目的是计算给定数据集中标记之间的共现，并利用这些共现来生成嵌入。

Vertex AI 提供了一个用于使用 Swivel 进行训练的流水线模板，因此您无需设计自己的流水线或编写自己的训练代码。

### 目标

在这份笔记本中，您将学习如何使用 Vertex AI Pipelines 训练自定义嵌入，并随后训练和部署一个向量搜索索引，使用这些嵌入。

本教程使用以下谷歌云 ML 服务：

- `Vertex AI Swivel` 内置算法
- `Vertex AI Vector Search`
- `Vertex AI Batch Prediction`

执行的步骤包括：

1. 训练`Swivel`算法以生成数据集的嵌入（编码器）。
2. 在训练过的编码器上进行示例预测（嵌入）。
3. 使用训练过的`Swivel`内置算法生成嵌入。
4. 将嵌入存储为`Vector Search`支持的格式。
5. 为嵌入创建一个`Vector Search Index`。
6. 将`Vector Search Index`部署到一个`Index Endpoint`。
7. 发出一个向量搜索预测请求。

### 数据集

本教程使用公共存储桶 `gs://cloud-samples-data/vertex-ai/matching-engine/swivel` 中的 `movielens示例数据集`，该数据集是从 [MovieLens电影评分数据集](https://grouplens.org/datasets/movielens/100k/) 生成的。该数据集经过处理，使得每一行包含了同一用户评分相同的电影。该目录还包括 `movies.csv`，其中映射了电影ID及其名称。

### 费用

本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- Dataflow
- Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage
定价](https://cloud.google.com/storage/pricing) 和 [Dataflow 定价](https://cloud.google.com/dataflow/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
基于预期使用情况生成费用估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行本笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud的[配置Python开发环境指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一个简明的指南：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端中的命令行运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端中的命令行运行`jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开此笔记本。

安装必需的软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade pip -q
! pip3 install {USER_FLAG} --upgrade scikit-learn -q
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform tensorboard-plugin-profile -q
! pip3 install {USER_FLAG} --upgrade tensorflow -q

### 重新启动内核

在安装附加包后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，以下步骤都是必须的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300的免费信用，可用于支付计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Dataflow API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,dataflow.googleapis.com)。

1. 如果您是在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 使用正确的项目来执行本笔记本中的所有命令。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

#### 设置您的项目 ID

**如果您不知道项目 ID**，您可以使用 `gcloud` 获取您的项目 ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

获取您的项目编号

现在项目ID已设置，您可以获得相应的项目编号。

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

#### 区域

您也可以更改`REGION`变量，该变量用于笔记本的其余部分操作。以下是Vertex AI支持的区域。我们建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果你正在参加直播教程会话，你可能正在使用共享的测试账号或项目。为避免用户在创建资源时发生名称冲突，你可以为每个实例会话创建一个时间戳，并将其附加到在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的谷歌云账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经得到认证。

**如果您正在使用Colab**，请运行下面的单元格，并按提示执行身份验证您的帐户通过oAuth。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到 [**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入名称，然后点击**创建**。

4. 在**授予此服务帐号访问项目**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

5. 点击*创建*。包含您密钥的JSON文件将下载到您的本地环境。

6. 将您的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入到下面的单元格中并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要按照以下步骤进行操作。**

当您使用云SDK提交内置的Swivel作业时，需要一个云存储桶来存储输入数据集和管道工件（训练好的模型）。

请在下方设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在的情况下才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 服务账户

**如果您不知道您的服务账户**，请尝试通过执行下方第二个单元格的gcloud命令来获取您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账号访问权限

运行以下命令，以授予您的服务账号访问权限，在您在上一步中创建的存储桶中读取和写入管道工件 -- 您只需要针对每个服务账号运行一次这些命令。

In [ ]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME

!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

导入库并定义常量
在本教程中定义使用的常量。

In [ ]:
SOURCE_DATA_PATH = "{}/swivel".format(BUCKET_URI)
PIPELINE_ROOT = "{}/pipeline_root".format(BUCKET_URI)

导入本教程中使用的软件包。

In [ ]:
import json

import pandas as pd
import tensorflow as tf
from google.cloud import aiplatform
from sklearn.metrics.pairwise import cosine_similarity

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存
     - `n1-highcpu`: 每个vCPU 0.9 GB内存
 - `vCPUs`: 数字范围为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

复制旋转模板

首先，下载提供的旋转模板和配置脚本。

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching-engine/swivel/pipeline/* .

### 设置您的流水线配置

更改您的流水线配置：

* pipeline_suffix: 流水线名称的后缀（允许小写字母和连字符）。
* machine_type: 例如 n1-standard-16。
* accelerator_count: 每台机器中的 GPU 数量。
* accelerator_type: 例如 NVIDIA_TESLA_P100，NVIDIA_TESLA_V100。
* region: 例如 us-east1（可选，默认为 us-central1）
* network_name: 例如，my_network_name（可选，否则将使用"default"网络）。

### VPC 网络对等连接、子网络和私有 IP 地址配置

执行以下单元格将生成两个文件：
1. `swivel_pipeline_basic.json`: 基本模板允许 Dataflow 作业使用公共 IP 和默认网络，并且不需要设置 VPC 网络对等连接用于 Vertex AI，**您将在此笔记本示例中使用它**。
1. `swivel_pipeline.json`: 该模板为 Dataflow 作业启用私有 IP 和子网配置，还需要设置 VPC 网络对等连接以用于 Vertex 自定义训练。该模板包括以下参数：
* "--subnetwork=regions/%REGION%/subnetworks/%NETWORK_NAME%",
* "--no_use_public_ips",
* \"network\": \"projects/%PROJECT_NUMBER%/global/networks/%NETWORK_NAME%\"

**警告** 为了指定私有 IP 和配置 VPC 网络，您需要在提交以下作业之前为您的子网（例如在"us-central1"上的"default"网络）[为 Vertex AI 设置 VPC 网络对等连接](https://cloud.google.com/vertex-ai/docs/general/vpc-peering#overview)。这是使用 DataFlow 和 Vertex AI 的私有 IP 地址所必需的。

In [ ]:
YOUR_PIPELINE_SUFFIX = "swivel-pipeline-movie"  # @param {type:"string"}
MACHINE_TYPE = "n1-standard-16"  # @param {type:"string"}
ACCELERATOR_COUNT = 2  # @param {type:"integer"}
ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"  # @param {type:"string"}

! chmod +x swivel_template_configuration*

! ./swivel_template_configuration_basic.sh -pipeline_suffix {YOUR_PIPELINE_SUFFIX} -project_number {PROJECT_NUMBER} -project_id {PROJECT_ID} -machine_type {MACHINE_TYPE} -accelerator_count {ACCELERATOR_COUNT} -accelerator_type {ACCELERATOR_TYPE} -pipeline_root {BUCKET_NAME}
! ./swivel_template_configuration.sh -pipeline_suffix {YOUR_PIPELINE_SUFFIX} -project_number {PROJECT_NUMBER} -project_id {PROJECT_ID} -machine_type {MACHINE_TYPE} -accelerator_count {ACCELERATOR_COUNT} -accelerator_type {ACCELERATOR_TYPE} -pipeline_root {BUCKET_NAME}

! sed "s/\\t/    /g" swivel_pipeline_basic.json > tmp.json
! mv tmp.json swivel_pipeline_basic.json

`swivel_pipeline_basic.json`和`swivel_pipeline.json`都已生成。

## 为 MovieLens 项目嵌入创建旋转工作

通过将编译后的规范传递给 `create_run_from_job_spec()` 方法来提交管道作业。请注意，您正在传递一个`parameter_values`字典，该字典指定要使用的管道输入参数。

以下表格显示了Swivel作业所需的运行时参数：

| 参数                     | 数据类型  | 描述                                                         | 是否需要               |
|----------------------------|----------|--------------------------------------------------------------------|------------------------|
| `embedding_dim`            | int      | 要训练的嵌入的维度。                         | 否 - 默认为100    |
| `input_base`         | string   | 存储输入数据的云存储路径。                       | 是                    |
| `input_type`               | string   | 输入数据的类型。可以是'text'（适用于维基百科样本）或'items'（适用于movielens样本）。   | 是                    |
| `max_vocab_size`               | int      | 生成嵌入的最大词汇量。                 | 否 - 默认为409600 |
| `num_epochs` | int | 训练的epoch数。 | 否 - 默认为20 |

简而言之，**items**输入类型意味着输入数据的每一行应为以空格分隔的项id。每行通过空格分隔进行分词。**text**输入类型意味着输入数据的每一行应等同于一个句子。每行通过小写化和空格分隔进行分词。

In [ ]:
# MovieLens items embedding sample

PARAMETER_VALUES = {
    "embedding_dim": 100,  # <---CHANGE THIS (OPTIONAL)
    "input_base": "{}/movielens_25m/train".format(SOURCE_DATA_PATH),
    "input_type": "items",  # For movielens sample
    "max_vocab_size": 409600,  # <---CHANGE THIS (OPTIONAL)
    "num_epochs": 5,  # <---CHANGE THIS (OPTIONAL)
}

### 将数据集复制到云存储

接下来，将MovieLens数据集复制到您的云存储存储桶中。

In [ ]:
# Copy the MovieLens sample dataset
! gsutil cp -r gs://cloud-samples-data/vertex-ai/matching-engine/swivel/movielens_25m/train/* {SOURCE_DATA_PATH}/movielens_25m

提交管道作业

接下来，将管道作业提交至 `Vertex AI Pipelines`。

In [ ]:
# Instantiate PipelineJob object
pl = aiplatform.PipelineJob(
    display_name=YOUR_PIPELINE_SUFFIX,
    # Whether or not to enable caching
    # True = always cache pipeline step result
    # False = never cache pipeline step result
    # None = defer to cache option for each pipeline component in the pipeline definition
    enable_caching=False,
    # Local or GCS path to a compiled pipeline definition
    template_path="swivel_pipeline_basic.json",
    # Dictionary containing input parameters for your pipeline
    parameter_values=PARAMETER_VALUES,
    # GCS path to act as the pipeline root
    pipeline_root=PIPELINE_ROOT,
)

# Submit the Pipeline to Vertex AI
# Optionally you may specify the service account below: submit(service_account=SERVICE_ACCOUNT)
# You must have iam.serviceAccounts.actAs permission on the service account to use it
pl.submit()

工作成功提交后，您可以查看其详细信息（包括您将在下面需要的运行名称）和日志。

### 使用TensorBoard检查模型

您可以使用TensorBoard来检查模型训练过程。为此，您需要找到训练模型工件的路径。在作业成功完成后（大约几个小时），您可以在[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction)浏览器中查看训练模型的输出路径。它将具有以下格式：

* {BUCKET_URI}/pipeline_root/{PROJECT_NUMBER}/swivel-{TIMESTAMP}/EmbTrainerComponent_-{SOME_NUMBER}/model/

您可以将此路径复制到下面的MODELOUTPUT_DIR中。

或者，您可以下载一个预训练模型到`{SOURCE_DATA_PATH}/movielens_model`并继续。该预训练模型仅用于演示目的，不适用于生产环境。

In [ ]:
! gsutil -m cp -r gs://cloud-samples-data/vertex-ai/matching-engine/swivel/models/movielens/model {SOURCE_DATA_PATH}/movielens_model

In [ ]:
SAVEDMODEL_DIR = os.path.join(SOURCE_DATA_PATH, "movielens_model/model")
LOGS_DIR = os.path.join(SOURCE_DATA_PATH, "movielens_model/tensorboard")

当训练开始时，您可以在TensorBoard中查看日志。

In [ ]:
# If on Vertex AI Workbench Notebooks, then don't execute this code.
if not os.getenv("DL_ANACONDA_HOME"):
    if "google.colab" in sys.modules:
        # Load the TensorBoard notebook extension.
        %load_ext tensorboard

In [ ]:
# If on Vertex AI Workbench Notebooks, then don't execute this code.
if not os.getenv("DL_ANACONDA_HOME"):
    if "google.colab" in sys.modules:
        %tensorboard --logdir $LOGS_DIR

对于**Vertex AI工作台笔记本**，您可以执行以下操作：

1. 从Google Cloud控制台打开Cloud Shell。
2. 安装依赖项：`pip3 install tensorflow tensorboard-plugin-profile`
3. 运行以下命令：`tensorboard --logdir {LOGS_DIR}`。您将看到输出消息“TensorBoard 2.x.0在http://localhost:<PORT>/（按CTRL+C退出）”。请注意端口号。
4. 您可以单击Web预览按钮，查看TensorBoard仪表板和性能分析结果。您需要将Web预览的端口配置为步骤3中收到的端口一样。

### 将模型上传至 `Vertex AI Model` 资源

首先，使用`upload()`方法导入模型，带有以下参数：

- `display_name`：模型资源的人类可读名称。
- `artifact_uri`：模型工件的云存储位置。
- `serving_container_image_uri`：部署容器。在本教程中，您将使用预构建的 Two-Tower 部署容器。

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest"

# Upload the trained model to Model resource
model = aiplatform.Model.upload(
    display_name="movies_" + TIMESTAMP,
    artifact_uri=SAVEDMODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

将模型部署到“Vertex AI Endpoint”

将“Vertex AI Model”资源部署到“Vertex AI Endpoint”以进行在线预测：

1. 创建一个“Endpoint”资源，为使用模型的用户暴露外部接口。
2. 在“Endpoint”准备就绪后，部署一个或多个模型实例到“Endpoint”。部署的模型运行 Swivel 编码器以提供嵌入。

有关在以下单元格中使用的 API 的更多信息，请参考 Vertex AI 预测指南中的 [使用 Vertex AI API 部署模型](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

创建一个`Vertex AI Endpoint`

接下来，您将创建一个`Vertex AI Endpoint`，之后您可以将您的`Vertex AI Model`资源部署到这个Endpoint上。

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="swivel_embedding_" + TIMESTAMP)

### 将`Model`资源部署到`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个终端点。每个部署的`Vertex AI Model`资源将拥有自己的用于提供二进制文件的部署容器。

在下面的示例中，您将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源。`Vertex AI Model`资源已经定义了用于部署容器的镜像。为了部署，您需要指定以下额外的配置设置：

- 机器类型。
- GPU 的类型和数量（如果有）。
- 静态、手动或自动扩展的 VM 实例。

在这个示例中，您将按照最少的指定参数来部署模型，如下：

- `model`：`Model`资源。
- `deployed_model_displayed_name`：部署模型实例的人类可读名称。
- `machine_type`：每个 VM 实例的机器类型。

由于资源配置的要求，这可能需要几分钟的时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name=DISPLAY_NAME,
    machine_type=DEPLOY_COMPUTE,
    traffic_split={"0": 100},
)

print(endpoint)

### 加载用于查询嵌入的电影ID和标题

接下来，下载电影ID和标题样本数据集，并将其读入 pandas Dataframe 中。

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching-engine/swivel/movielens_25m/movies.csv ./movies.csv

movies = pd.read_csv("movies.csv")
print(f"Movie count: {len(movies.index)}")
movies.head()

## 创建嵌入

现在您已经在`Vertex AI Prediction`上部署了编码器模型，您可以调用该模型为新数据生成嵌入。

### 使用SDK进行在线预测

[在线预测](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models) 用于在最小延迟下同步查询一小批实例的模型。以下函数使用Vertex AI SDK for Python 调用部署的模型。

BLAH 您想要在上预测嵌入的输入数据应提供为电影ID的列表。请注意，您还应为每个输入实例提供一个唯一的`key`字段（类型为str），以便您可以将每个输出嵌入与其对应的输入关联起来。

In [ ]:
# Change to your favourite movies.
query_movies = [
    "Lion King, The (1994)",
    "Aladdin (1992)",
    "Star Wars: Episode IV - A New Hope (1977)",
    "Star Wars: Episode VI - Return of the Jedi (1983)",
    "Terminator 2: Judgment Day (1991)",
    "Aliens (1986)",
    "Godfather, The (1972)",
    "Goodfellas (1990)",
]


def get_movie_id(title):
    return list(movies[movies.title == title].movieId)[0]


instances = [str(get_movie_id(title)) for title in query_movies]
print(instances)

### 为嵌入向量进行在线预测请求

接下来，使用`predict()`方法进行预测请求。

In [ ]:
predictions = endpoint.predict(instances=instances)
embeddings = predictions.predictions
print("Number of embeddings:", len(embeddings))
print(embeddings[0])

#### 探索电影嵌入

使用嵌入，探索样本电影列表中每部电影与其他电影的相似程度。

*注：* 1.0 的值表示它们具有相同的嵌入。

In [ ]:
for idx1 in range(0, len(input_items) - 1, 2):
    item1 = instances[idx1]
    title1 = query_movies[idx1]
    print(title1)
    print("==================")
    embedding1 = embeddings[idx1]
    for idx2 in range(0, len(instances)):
        item2 = input_items[idx2]
        embedding2 = embeddings[idx2]
        similarity = round(cosine_similarity([embedding1], [embedding2])[0][0], 5)
        title1 = query_movies[idx1]
        title2 = query_movies[idx2]
        print(f" - Similarity to '{title2}' = {similarity}")
    print()

您可以使用[TensorBoard嵌入式项目管理器](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin)来图形化表示高维嵌入，这有助于检查和理解您的嵌入。

使用`gcloud`命令行界面也可以进行在线预测。

In [ ]:
import json

request = json.dumps({"instances": input_items})
with open("request.json", "w") as writer:
    writer.write(f"{request}\n")

ENDPOINT_ID = endpoint.resource_name

! gcloud ai endpoints predict {ENDPOINT_ID} \
  --region={REGION} \
  --json-request=request.json

### 进行批量预测

[批量预测](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions) 用于异步地对一批输入数据进行预测。如果您有大量输入数据且不需要立即的响应，比如获取候选对象的嵌入以便为最近邻搜索服务（如 [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/matching-engine/overview)）创建索引，建议使用此功能。

### 创建批量输入文件

接下来，您需要生成批量输入文件，以为数据集生成嵌入，然后使用 `Vertex AI Vector Search` 创建一个索引。在本例中，数据集包含200000个唯一标识符（1...200000）。您将使用训练过的编码器为每个唯一标识符生成一个预测的嵌入。

输入数据需要存储在Cloud Storage中，并且需要采用JSONL格式。您可以使用下面提供的示例查询对象文件。与在线预测类似，建议包含`key`字段，以便将每个输出嵌入与相应的输入进行关联。

In [ ]:
QUERY_EMBEDDING_PATH = f"{BUCKET_URI}/embeddings/train.jsonl"

with tf.io.gfile.GFile(QUERY_EMBEDDING_PATH, "w") as f:
    for i in range(1, 200001):
        query = str(i)
        f.write(json.dumps(query) + "\n")

print("\nNumber of embeddings: ")
! gsutil cat {QUERY_EMBEDDING_PATH} | wc -l

In [ ]:
! gsutil cat {QUERY_EMBEDDING_PATH} | head

### 发送预测请求

要进行批量预测请求，请调用模型对象的`batch_predict`方法，并使用以下参数：
- `instances_format`：批量预测请求文件的格式："jsonl"，"csv"，"bigquery"，"tf-record"，"tf-record-gzip"或"file-list"
- `prediction_format`：批量预测响应文件的格式："jsonl"，"csv"，"bigquery"，"tf-record"，"tf-record-gzip"或"file-list"
- `job_display_name`：预测任务的人类可读名称。
- `gcs_source`：存储批量预测请求的一个或多个云存储路径的列表。
- `gcs_destination_prefix`：服务将写入预测结果的云存储路径。
- `model_parameters`：用于提供预测结果的额外过滤参数。
- `machine_type`：用于训练的计算机类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作机器副本的加速器数量。
- `starting_replica_count`：最初要启动的计算实例数量。
- `max_replica_count`：要扩展至的计算实例的最大数量。在本教程中，只会启动一个实例。

### 计算实例扩展

您可以指定一个单一实例（或节点）来处理您的批量预测请求。本教程使用单个节点，所以变量`MIN_NODES`和`MAX_NODES`都设置为`1`。

如果您希望使用多个节点来处理批量预测请求，请将`MAX_NODES`设置为您要使用的最大节点数量。Vertex AI会自动调整用于提供预测的节点数量，最多达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)来了解使用多个节点进行自动扩展的成本。

In [ ]:
MIN_NODES = 1
MAX_NODES = 4

batch_predict_job = model.batch_predict(
    job_display_name="batch_predict_swivel",
    gcs_source=[QUERY_EMBEDDING_PATH],
    gcs_destination_prefix=f"{BUCKET_URI}/embeddings/output",
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

### 获取预测的嵌入向量

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您可以调用`iter_outputs()`方法来获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

result_files = []
for prediction_result in prediction_results:
    result_file = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    result_files.append(result_file)

print(result_files)

将嵌入以JSONL格式保存

接下来，您将预测的嵌入作为一个经过JSONL格式化的文件进行存储。每个嵌入都存储为：

{ 'id': .., 'embedding': [ ... ] }

用于索引的嵌入格式可以是CSV、JSON或Avro格式中的任何一种。

了解更多关于[嵌入格式用于索引](https://cloud.google.com/vertex-ai/docs/matching-engine/using-matching-engine#json)

In [ ]:
embeddings = []
for result_file in result_files:

    with tf.io.gfile.GFile(result_file, "r") as f:
        instances = list(f)

    for instance in instances:
        instance = instance.replace('\\"', "'")
        result = json.loads(instance)
        prediction = result["prediction"]
        key = result["instance"]

        embedding = {"id": key, "embedding": prediction}
        embeddings.append(embedding)

print("Number of embeddings", len(embeddings))
print("Encoding Dimensions", len(embeddings[0]["embedding"]))
print("Example embedding", embeddings[0])

with open("embeddings.json", "w") as f:
    for i in range(len(embeddings)):
        f.write(json.dumps(embeddings[i]).replace('"', "'"))
        f.write("\n")

! head -n 2 embeddings.json

将JSONL格式的嵌入存储在云存储中

接下来，您将训练数据上传到您的云存储桶中。

In [ ]:
EMBEDDINGS_URI = f"{BUCKET_URI}/embeddings/swivel/"
! gsutil cp embeddings.json {EMBEDDINGS_URI}

### 创建向量搜索索引

接下来，您需要为您的嵌入创建索引。目前支持两种索引算法：

- `create_tree_ah_index()`: 浅树 + 非对称哈希。
- `create_brute_force_index()`: 线性搜索。

在本教程中，您将在生产规模中使用`create_tree_ah_index()`方法。以下是调用该方法时所需的参数：

- `display_name`: 索引的可读名称。
- `contents_delta_uri`: 嵌入的云存储位置，可以是待插入、更新或删除的嵌入。
- `dimensions`: 输入向量的维数。
- `approximate_neighbors_count`: (对于Tree AH) 在执行精确重新排序之前通过近似搜索查找的默认邻居数。精确重新排序是指通过更昂贵的距离计算方法重新排序近似搜索算法返回的结果。
- `distance_measure_type`: 用于最近邻搜索的距离度量方法。
    - `SQUARED_L2_DISTANCE`: 欧氏距离 (L2)
    - `L1_DISTANCE`: 曼哈顿距离 (L1)
    - `COSINE_DISTANCE`: 余弦距离。定义为1-余弦相似度。
    - `DOT_PRODUCT_DISTANCE`: 默认值。定义为点积的负数。
- `description`: 索引的描述。
- `labels`: 用户元数据，以字典形式提供。
- `leaf_node_embedding_count`: 每个叶子节点上的嵌入数量。如果未设置，默认值为1000。
- `leaf_nodes_to_search_percent`: 任何查询可能搜索的叶节点的默认百分比。必须在1-100的范围内，包括1和100。如果未设置，默认值为10 (表示10%)。

此过程最多可能需要30分钟。

了解有关[配置向量搜索索引](https://cloud.google.com/vertex-ai/docs/matching-engine/configuring-indexes)的更多信息。

In [ ]:
DIMENSIONS = len(embeddings[0]["embedding"])
DISPLAY_NAME = "movies"

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=50,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Two tower generated embeddings",
    labels={"label_name": "label_value"},
    # TreeAH specific parameters
    leaf_node_embedding_count=100,
    leaf_nodes_to_search_percent=7,
)

INDEX_RESOURCE_NAME = tree_ah_index.resource_name
print(INDEX_RESOURCE_NAME)

## 设置VPC对等连接网络

为了使用`矢量搜索索引`，您需要在您的项目和`Vertex AI矢量搜索`服务项目之间设置VPC对等连接网络。这可以消除网络流量中的额外跳跃，并允许使用高效的gRPC协议。

了解更多有关[VPC对等连接](https://cloud.google.com/vertex-ai/docs/general/vpc-peering)的信息。

**重要提示：每个项目只能设置一个到servicenetworking.googleapis.com的VPC对等连接。**

### 为默认网络创建VPC对等连接

为简单起见，我们设置了与默认网络的VPC对等连接。您可以为您的项目创建不同的网络。

如果您与任何其他网络设置VPC对等连接，请确保该网络已存在并且您的虚拟机正在该网络上运行。

In [ ]:
# This is for display only; you can name the range anything.
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"
NETWORK = "default"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 will be
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### 创建VPC连接

接下来，创建VPC对等连接。

*注意:* 如果出现“PERMISSION DENIED”，可能是因为您的默认服务帐户没有设置必要的角色'计算网络管理员'。 在云控制台中，执行以下步骤。

1. 进入`IAM & Admin`。
2. 找到您的服务帐户。
3. 点击编辑图标。
4. 选择`添加另一个角色`。
5. 输入'Compute Network Admin'。
6. 选择`保存`。

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

检查您对等连接的状态。

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

构建完整的网络名称

在随后为VPC peering创建`Vector Search Index Endpoint`资源时，您需要拥有完整的网络资源名称。

In [ ]:
full_network_name = f"projects/{PROJECT_NUMBER}/global/networks/{NETWORK}"

### 在VPC网络中创建IndexEndpoint

接下来，您创建一个`向量搜索索引终端点`，类似于在对等网络上创建`私有终端点`进行预测的概念。

要创建`Index Endpoint`资源，您需使用以下参数调用`create()`方法：

- `display_name`：`Index Endpoint`的可读名称。
- `description`：`Index Endpoint`的描述。
- `network`：VPC网络资源名称。

In [ ]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="index endpoint description",
    network=full_network_name,
)

INDEX_ENDPOINT_NAME = index_endpoint.resource_name
print(INDEX_ENDPOINT_NAME)

将`向量搜索索引`部署到`索引终端`资源

接下来，使用`deploy_index()`方法和以下参数将您的索引部署到`索引终端`：

- `display_name`：部署索引的可读名称。
- `index`：您的索引。
- `deployed_index_id`：部署索引的用户分配的标识符。
- `machine_type`：（可选）VM实例类型。
- `min_replica_count`：（可选）自动扩展的最小VM实例数。
- `max_replica_count`：（可选）自动扩展的最大VM实例数。

了解有关[索引终端的机器资源](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.indexEndpoints#DeployedIndex)的更多信息。

In [ ]:
DEPLOYED_INDEX_ID = "tree_ah_twotower_deployed_" + TIMESTAMP

MIN_NODES = 1
MAX_NODES = 2
DEPLOY_COMPUTE = "n1-standard-16"

index_endpoint.deploy_index(
    display_name="deployed_index_for_demo",
    index=tree_ah_index,
    deployed_index_id=DEPLOYED_INDEX_ID,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

print(index_endpoint.deployed_indexes)

### 创建和执行在线查询

现在您的索引已部署，您可以进行查询。

首先，使用合成数据构建一个向量 `query`，以用作返回匹配项的示例。

接下来，使用 `match()` 方法进行匹配请求，参数如下：

- `deployed_index_id`：部署索引的标识符。
- `queries`：查询（实例）的列表。
- `num_neighbors`：要返回的最接近匹配项的数量。

In [ ]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

# Test query
queries = [embeddings[0]["embedding"], embeddings[1]["embedding"]]

matches = index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=queries, num_neighbors=NUM_NEIGHBOURS
)

for instance in matches:
    print("INSTANCE")
    for match in instance:
        print(match)

## 清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Delete endpoint resource
endpoint.delete(force=True)

# Delete model resource
model.delete()

# Force undeployment of indexes and delete endpoint
try:
    index_endpoint.delete(force=True)
except Exception as e:
    print(e)

# Delete indexes
try:
    tree_ah_index.delete()
except Exception as e:
    print(e)

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $OUTPUT_DIR